<a href="https://colab.research.google.com/github/Johny85/World-of-Scripts/blob/master/Python%20Secure%20Login.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### PYTHON Secure Login System ###

In [ ]:
### Middle Tier ###

import sqlite3
import hashlib

conn = sqlite3.connect("Main.db")
cur = conn.cursor()

cur.execute("""
Create Table if not exists Profile (
    id integer primary key,
    username varchar not null,
    password varchar not null
)
""")

username1, password1 = "John", hashlib.sha256("password".encode()).hexdigest()
username2, password2 = "TripleHHH", hashlib.sha256("password".encode()).hexdigest()
username3, password3 = "Randy", hashlib.sha256("password".encode()).hexdigest()
username4, password4 = "Undertaker", hashlib.sha256("password".encode()).hexdigest()
username5, password5 = "Batista", hashlib.sha256("password".encode()).hexdigest()

cur.execute("Insert into Profile (username, password) values(?, ?)", (username1, password1))
cur.execute("Insert into Profile (username, password) values(?, ?)", (username2, password2))
cur.execute("Insert into Profile (username, password) values(?, ?)", (username3, password3))
cur.execute("Insert into Profile (username, password) values(?, ?)", (username4, password4))
cur.execute("Insert into Profile (username, password) values(?, ?)", (username5, password5))

conn.commit()

In [ ]:
### Server Side Script ###

import sqlite3
import hashlib
import socket
import threading

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(("localhost", 9999))

server.listen()

def handle_conn(c):
    c.send("Username: ".encode())
    username = c.recv(1024).decode()
    c.send("Password: ".encode())
    password = c.recv(1024)
    password = hashlib.sha256(password).hexdigest()

    conn = sqlite3.connect("Main.db")
    cur = conn.cursor()

    cur.execute("Select * from Profile where username = ? and password = ?", (username, password))

    if cur.fetchall():
        c.send("Login Successful!". encode())
    else:
        c.send("Login Failed!". encode())


while True:
    client, addr = server.accept()
    threading.Thread(target=handle_conn, args=(client,)).start()

In [ ]:
### Client Side Script ###

import socket

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(("localhost", 9999))

message = client.recv(1024).decode()
client.send(input(message).encode())
message = client.recv(1024).decode()
client.send(input(message).encode())

print(client.recv(1024).decode())